In [1]:
import os, sys
sys.path.append("/home/files/feature_orthogonality")
from utils.session_config import setup_gpus

In [2]:
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
setup_gpus(memory_fraction=0.2)

1 Physical GPUs, 1 Logical GPUs


# Load mnist images

In [4]:
data_0 = np.load("0.npy")
label_0 = np.zeros(len(data_0), np.int32)
data_5 = np.load("5.npy")
label_5 = np.ones(len(data_5), np.int32)
data_7 = np.load("7.npy")
label_7 = np.ones(len(data_7), np.int32)*2

In [5]:
train_x = tf.concat([data_0, data_5, data_7], axis=0)
train_x = tf.cast(train_x, tf.float32)
train_x = train_x[:,:,:,tf.newaxis]/255.
train_y = tf.concat([label_0, label_5, label_7], axis=0)

In [6]:
print(train_x.shape)
print(train_y.shape)

(17609, 28, 28, 1)
(17609,)


In [7]:
#(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
# train_x = train_x[:,:,:,tf.newaxis]/255.
# test_x = test_x[:,:,:,tf.newaxis]/255.
# print(train_x.shape)
# print(test_x.shape)

# Build Model

In [7]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

model = tf.keras.Sequential()
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))


In [8]:
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])
model.fit(train_x, train_y, epochs=10)

Epoch 1/10
551/551 [==============================] - 1s 2ms/step - loss: 0.0856 - acc: 0.9718
Epoch 2/10
551/551 [==============================] - 1s 2ms/step - loss: 0.0118 - acc: 0.9960
Epoch 3/10
551/551 [==============================] - 1s 2ms/step - loss: 0.0051 - acc: 0.9981
Epoch 4/10
551/551 [==============================] - 1s 2ms/step - loss: 0.0046 - acc: 0.9985
Epoch 5/10
551/551 [==============================] - 1s 2ms/step - loss: 0.0026 - acc: 0.9990
Epoch 6/10
551/551 [==============================] - 1s 2ms/step - loss: 0.0018 - acc: 0.9995
Epoch 7/10
551/551 [==============================] - 1s 2ms/step - loss: 0.0023 - acc: 0.9991
Epoch 8/10
551/551 [==============================] - 1s 2ms/step - loss: 0.0020 - acc: 0.9992
Epoch 9/10
551/551 [==============================] - 1s 2ms/step - loss: 0.0011 - acc: 0.9997
Epoch 10/10
551/551 [==============================] - 1s 2ms/step - loss: 4.0680e-04 - acc: 0.9998


In [9]:
model.layers

In [14]:
model.layers[-2].get_weights()[1]

array([-0.00837162,  0.01268043, -0.01906341,  0.00834394,  0.02072056,
        0.01945855, -0.00402998,  0.02504837,  0.0271862 , -0.00940461,
       -0.03340371,  0.0162708 , -0.03813823, -0.00859336,  0.01771883,
       -0.00686041,  0.01806572, -0.038187  , -0.00607036, -0.00743724,
       -0.0086609 , -0.01248649, -0.00616797,  0.01870752, -0.0071583 ,
        0.02267759,  0.01688812,  0.02647537,  0.0142962 ,  0.01497921,
       -0.00600793,  0.01715473], dtype=float32)

In [15]:
model.fit(train_x, train_y, epochs=1)

551/551 [==============================] - 1s 2ms/step - loss: 2.1446e-04 - acc: 0.9999


In [26]:
model.layers[-2].get_weights()[0]

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [10]:
new_model = tf.keras.Model(model.input, model.layers[-2].output)

In [11]:
new_model.layers

In [22]:
new_model.layers[-1].get_weights()[0]

(1600, 32)

In [23]:
new_model.layers[-1].set_weights([tf.ones([1600,32], tf.float32), tf.ones([32], tf.float32)])

In [24]:
new_model.layers[-1].get_weights()[1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

In [17]:
new_model.layers[-1].get_weights()[1]

array([-0.00837162,  0.01362524, -0.01852702,  0.00769823,  0.02287077,
        0.02361904, -0.00402998,  0.02381487,  0.02467142, -0.00742251,
       -0.03340371,  0.01672162, -0.03791286, -0.00859336,  0.01908844,
       -0.00686041,  0.01770509, -0.038187  , -0.00607036, -0.00743724,
       -0.0086609 , -0.01248649, -0.00616797,  0.01930858, -0.0071583 ,
        0.0224361 ,  0.01676522,  0.02594074,  0.01343621,  0.01536536,
       -0.00600793,  0.01591624], dtype=float32)

In [18]:
new_model.save("temp.h5")

In [20]:
newnew_model = tf.keras.models.load_model("temp.h5")
newnew_model.layers

# Visualization

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
def get_weight_cosine_similarity(weight):
    w = weight
    if len(w.shape) == 4: #convolution
        H,W,C,N = w.shape
        row_dims = H*W*C
        col_dims = N
    else: #dense
        D,N = w.shape
        row_dims = D
        col_dims = N
    w = tf.reshape(w, (row_dims, col_dims))
    norm = tf.norm(w, axis=0)
    w = w/norm #normalize
    wT = tf.transpose(w)
    correlations = tf.matmul(wT,w)
    return correlations

def get_weight_l2norm(weight):
    w = weight
    if len(w.shape) == 4: #convolution
        H,W,C,N = w.shape
        row_dims = H*W*C
        col_dims = N
    else: #dense
        D,N = w.shape
        row_dims = D
        col_dims = N
    w = tf.reshape(w, (row_dims, col_dims))
    norm = tf.norm(w, axis=0)
    return norm

In [ ]:
weight = model.get_layer("conv2d").get_weights()[0]
correlations = get_weight_cosine_similarity(weight)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 800
sns.heatmap(correlations, vmin=-1, vmax=1, cmap='RdBu_r', center=0, annot=True, fmt='.1f',xticklabels=False, yticklabels=False,annot_kws={"size": 4})

In [ ]:
norm = get_weight_l2norm(weight)
plt.bar(range(correlations.shape[0]), norm)
plt.title('Norm of weights', fontsize=20)
plt.show()

In [ ]:
norm